In [4]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from skvideo.measure import niqe, viideo_score, viideo_features, videobliinds_features, brisque_features
from skvideo.motion import globalEdgeMotion, blockMotion

import helper_functions as hf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, \
    Dropout, Conv2D, MaxPooling2D, TimeDistributed, LSTM, LeakyReLU, Average, Lambda, K
from keras.utils import to_categorical

In [5]:
def read_data():
    X_train = hf.import_video_data(158, "input/train/")
    X_test = hf.import_video_data(69, "input/test/")

    test_index = np.arange(69)

    y_train, _ = hf.read_csv_to_matrix("train_target.csv", "id")
    return X_train, X_test, np.squeeze(y_train), test_index

In [6]:
print("==> Reading data")
X_train, X_test, y_train, test_index = read_data()

==> Reading data


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'input/train/0.avi'